# NS 方程

Navier-Stocks 方程:
\begin{equation}
\left\{
\begin{aligned}
&\partial_t u - \mu\Delta u + (u\cdot\nabla)u + \nabla p = f, && {\rm in} \quad \Omega\times(0, T]\\
&\nabla\cdot u = 0, && {\rm in} \quad \Omega\times(0, T]
\end{aligned}
\right.
\end{equation}

初边值条件
\begin{equation}
\left\{
\begin{aligned}
& u = 0, && {\rm on} \quad \partial\Omega\times(0, T] \\
&u_0 = (y, -x) &&{\rm in} \quad \Omega\quad{\rm at}\quad t = 0\\
\end{aligned}
\right.
\end{equation}


In [ ]:
from firedrake import *

mu = 1
T = 0.25

N_S = 16
N_T = 128

tau = T/N_T
h = 1/N_S

mesh = RectangleMesh(N_S, N_S, 1, 1)

x = SpatialCoordinate(mesh)
# u_0 = as_vector((x[1] - 0.5, - x[0] + 0.5))
u_0 = as_vector((x[1], - x[0]))
f = as_vector([0, -1])

## 函数空间

采用 MINI 元, 即 P1 $\times$ P1b.

P1b 由 P1 加上 Bubble 组成.

`NodalEnrichedElement`, `EnrichedElement`

`VectorFunctionSpace` 构造向量空间

In [ ]:
cell = mesh.ufl_cell()
tdim = cell.topological_dimension()

# Mini element: P1 X P1b
P1 = FiniteElement("CG", cell, 1)
B = FiniteElement("B", cell, tdim+1)
P1b = P1 + B # or P1b = NodalEnrichedElement(P1, B)

V_u = VectorFunctionSpace(mesh, P1b)
V_p = FunctionSpace(mesh, "CG", 1)
V = MixedFunctionSpace([V_u, V_p])

## 弱形式

\begin{equation}
\left\{
\begin{aligned}
&\frac{1}{\tau}(u^n - u^{n-1}, v) + \mu(\nabla u^n, \nabla v) + ((u^n\cdot\nabla)u^n, v) - (p^n, \nabla\cdot v) = (f^n, v)\\
&(q, \nabla\cdot u^n) = 0
\end{aligned}
\right.
\end{equation}

+ `TrialFunctions`, `TestFunctions`: 

  以 `tuple` 返回函数空间中的试验/测试函数,

  主要用于 `MixedFunctionSpace`.
  
+ `split`
    + `split`: 以索引的方式获取 `MixedFunctionSpace` 中函数的分量 (保留 UFL 关联信息, 用于定义变分形式)

由于该问题是非线性问题, 我们打算用 `NonlinearVariationalSolver` 进行求解, 所以下面定义 `w` 使用了 `Function` 而不是 `TrialFunction`/`TrialFunctions`.

In [ ]:
w = Function(V) # u and p
u, p = split(w)

v, q = TestFunctions(V)

w_nm1 = Function(V)
u_nm1, p_nm1 = w_nm1.subfunctions
u_nm1.rename('u_h') # for visualization in paraview
p_nm1.rename('p_h')

Re = Constant(mu)

F = \
      Constant(1/tau)*inner(u - u_nm1, v)*dx \
    + Re*inner(grad(u+u_nm1)/2, grad(v))*dx \
    + inner(dot(grad(u), (u+u_nm1)/2), v)*dx \
    - p*div(v)*dx \
    + div(u)*q*dx \
    - inner(f, v)*dx

## 定义 Solver

类似于纯 Neumann 问题, 我们将使用 `nullspace` 参数.

注意下面混合空间中, 边界条件和 `nullspace` 的定义.

In [ ]:
bc = DirichletBC(V.sub(0), 0, 'on_boundary')
nullspace = MixedVectorSpaceBasis(V, [V.sub(0), VectorSpaceBasis(constant=True, comm=mesh.comm)])

problem = NonlinearVariationalProblem(F, w, bcs=bc)  # F = 0
solver = NonlinearVariationalSolver(problem,
                                    options_prefix='ns',
                                    solver_parameters=None, # {'snes_converged_reason': None, 'snes_max_it': 100},
                                    nullspace=nullspace
                                   )

## 时间循环和保存结果到 `pvd` 文件

In [ ]:
u_, p_ = w.subfunctions

output = File('pvd/ns-equation.pvd')

u_nm1.project(u_0)
output.write(u_nm1, p_nm1, time=0)

for i in ProgressBar('T', bar_prefix=" |", bar_suffix="| ", empty_fill=" ", fill="#").iter(range(N_T)):
    t = tau*(i+1)
    
    solver.solve()
    
    u_nm1.assign(u_)
    p_nm1.assign(p_)

    (i+1)%32 or output.write(u_nm1, p_nm1, time=t)

T |##################################################| 128/128 [0:00:01]


### Constant 用于时间依赖的表达式

In [ ]:
from firedrake import *
mesh = RectangleMesh(10, 10, 1, 1)
C1 = Constant(0)

x, y = SpatialCoordinate(mesh)
expr = C1*(x+y)

v = []
for i in range(5):
    t = i*0.1
    C1.assign(t)
    v.append(
        assemble(expr*dx)
    )

print(v)

[0.0, 0.0999999999999999, 0.1999999999999998, 0.2999999999999997, 0.3999999999999996]


## ParaView 可视化计算结果

Pipeline 和 Filter

### 二维结果 (surf 图)

Filter: Wrap by scalar

### 选择部分区域显示

View -> Find Data



### 并行数据显示各进程区域

Filter -> Connectivity